In [11]:
import csv
import re
import scipy
import requests
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from scipy import stats
from io import StringIO

# Graph of sales by product

In [12]:
def processCSV(data_name: str):
    # Send a GET request to the endpoint
    response = requests.get(f"http://127.0.0.1:8000/{data_name}")

        # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Create a CSV writer
        output_csv = StringIO()
        csv_writer = csv.writer(output_csv)

        # Split input string into lines and write them to the CSV
    for line in response.text.split('\\n'):
        csv_writer.writerow(line.split(','))
        print(line)
        # Get the CSV content as a string
        csv_content = output_csv.getvalue().replace("\"", "")

        # Close the StringIO object
        output_csv.close()

        df = pd.read_csv(StringIO(csv_content))

        return df
    else:
        print("Failed to retrieve data. Status code:", response.status_code)


products = processCSV("product")
print(products)

products = products[['product_id', 'product_group', 'product_category', 'product_type', 'product']]

customers = processCSV("customers")
customers = customers[['customer_id', 'gender', 'birth_year']]

sales_receipts = processCSV("sales_reciepts")

specific_date = '2019-04-01' # Filtering for Date
specific_instore = 'N' # Filtering for Instore

sales_receipts_final = sales_receipts.loc[
    (sales_receipts['transaction_date'].str.contains(specific_date)) &
    (sales_receipts['instore_yn'] == specific_instore)
]

filtered_sales = sales_receipts_final[['product_id', 'transaction_date', 'customer_id', 'quantity']]
sales_receipts_grouped = filtered_sales.groupby('product_id')['quantity'].sum().reset_index()
sales_receipts_final = pd.merge(sales_receipts_grouped, products, on='product_id', how='left')
sales_receipts_final.sort_values(by="product_id", ascending=False)

highest_product_id = sales_receipts_final['product_id'].max()
lowest_product_id = sales_receipts_final['product_id'].min()

bar_x_labels = []; # Labels for X axis
for id, product in zip(sales_receipts_final['product_id'], sales_receipts_final['product_type']):
    bar_x_labels.append(f"{id} - {product}")

fig, ax = plt.subplots(figsize=(20, 10))
plt.bar(sales_receipts_final['product_id'], sales_receipts_final['quantity'], color=(0.2, 0.6, 0.2, 1))
plt.xticks(sales_receipts_final['product_id'], bar_x_labels, rotation=90)

ax.set_xlabel(f"In store: {'No' if specific_instore == 'N' else 'Yes'}\n Date: {specific_date}")
ax.set_ylabel("Purchased quantity", color='blue')

for id, quan in zip(sales_receipts_final['product_id'], sales_receipts_final['quantity']):
    ax.text(id - .30, quan / 2, str(quan), color='black', fontweight='bold', rotation=90)

"""product_id"//product_group//product_category//product_type//product//product_description//unit_of_measure//current_wholesale_price//current_retail_price//tax_exempt_yn//promo_yn//new_product_yn

Empty DataFrame
Columns: ["product_id//product_group//product_category//product_type//product//product_description//unit_of_measure//current_wholesale_price//current_retail_price//tax_exempt_yn//promo_yn//new_product_yn]
Index: []


KeyError: "None of [Index(['product_id', 'product_group', 'product_category', 'product_type',\n       'product'],\n      dtype='object')] are in the [columns]"